#Imports

In [1]:
#Instalaciones para descargar las imágenes al entorno
!pip install gdown --upgrade --no-cache-dir
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [53]:
#Imports
from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd
import gdown
import io
import os
from googleapiclient.http import MediaIoBaseDownload

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import shutil
import random
from PIL import Image
from torchvision import transforms, datasets, models

import torch.nn as nn
import tensorflow as tf
import torch.nn.functional as F

import torch
import torch.optim as optim

import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Crear dataset

In [36]:
#Preparar el entorno para descargar los archivos
auth.authenticate_user()
drive_service = build('drive', 'v3')

#Links de acceso público a las carpetas con las imágenes
folder_ia = '14xM8jjGl7QPMDUkti6iUl4M1m6qcs4L1'
folder_real = '1ruvbcczrUZ2W6rmmC7TvLUQxtOD_Cg3J'

#Función para descargar las imágenes al entorno
def download_files_from_folder(folder_id, output_folder, label):
  query = f"'{folder_id}' in parents"
  results = drive_service.files().list(q=query, fields='nextPageToken, files(id, name)').execute()
  items = results.get('files', [])

  if not items:
    print('No hay archivos en la carpeta.')
    return

  carpeta_label = os.path.join(output_folder, label)
  os.makedirs(carpeta_label, exist_ok=True)

  for item in items:
    file_id = item['id']
    file_name = item['name']
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
      status, done = downloader.next_chunk()
    with open(os.path.join(carpeta_label, file_name), 'wb') as f:
      f.write(fh.getbuffer())

os.makedirs('dataset', exist_ok=True)

download_files_from_folder(folder_ia, 'dataset', 'ia')
print(f"Carpeta fotos_ia completada.")
download_files_from_folder(folder_real, 'dataset', 'real')
print(f"Carpeta fotos_real completada.")

Carpeta fotos_ia completada.
Carpeta fotos_real completada.


# Preparación de datos

In [37]:
#Preparación de las imágenes

#Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  #Cambiar resolución de imagen
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) #Normalizar los píxeles
])

data = datasets.ImageFolder('dataset', transform=transform)

#Comprobar labels
print(f"Labels: {data.classes}")
print(f"Mapping de labels: {data.class_to_idx}")

Labels: ['ia', 'real']
Mapping de labels: {'ia': 0, 'real': 1}


In [39]:
# Separación de sets
train_size = int(len(data)*0.7) # 70% de contenido para el training set
val_size = int(0.15 * len(data)) # 15% de contenido para el validation set
test_size = int(len(data) - train_size - val_size) # 15% de contenido para el test set

train, val, test = random_split(data, [train_size, val_size, test_size])

# Crear los DataLoaders
train_loader = DataLoader(train, batch_size=32, shuffle=True)
val_loader = DataLoader(val, batch_size=32, shuffle=False)
test_loader = DataLoader(test, batch_size=32, shuffle=True)

# Creación de modelos

In [41]:
#Crear los modelos

#Resnet18 (pretrained)
modelo_resnet = models.resnet18(pretrained=True)
modelo_resnet.fc = nn.Linear(modelo_resnet.fc.in_features, 2)

#MobileNetV2 (pretrained)
modelo_mobilenet = models.mobilenet_v2(pretrained=True)
modelo_mobilenet.classifier[1] = nn.Linear(modelo_mobilenet.classifier[1].in_features, 2)

#CNN
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    #Convolutional layers
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

    #Fully connected layers
    self.fc1 = nn.Linear(128 * 28 * 28, 512)
    self.fc2 = nn.Linear(512, 2)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))

    #Flatten
    x = x.view(-1, 128 * 28 * 28)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)

    return x

modelo_cnn = CNN()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-d

In [44]:
# Optimizar modelos
criterion = nn.CrossEntropyLoss()
resnet_optimizer = torch.optim.Adam(modelo_resnet.parameters(), lr=0.001)
mobilenet_optimizer = torch.optim.Adam(modelo_mobilenet.parameters(), lr=0.001)
cnn_optimizer = torch.optim.Adam(modelo_cnn.parameters(), lr=0.001)

In [45]:
# Preparar el device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelo_resnet = modelo_resnet.to(device)
modelo_mobilenet = modelo_mobilenet.to(device)
modelo_cnn = modelo_cnn.to(device)

# Entrenar modelos

In [56]:
def evaluation(model, dataLoader, device):
  # Poner modelo en modo evaluación
  model.eval()

  preds = []
  labels = []

  with torch.no_grad():
    for input, label in dataLoader:
      input = input.to(device)
      label = label.to(device)

      output = model(input)
      _, pred = torch.max(output, 1)

      preds.extend(pred.cpu().numpy())
      labels.extend(label.cpu().numpy())

  # Calcular métricas
  accuracy = accuracy_score(labels, preds)
  precision = precision_score(labels, preds, average='weighted')
  recall = recall_score(labels, preds, average='weighted')
  f1 = f1_score(labels, preds, average='weighted')
  matrix = confusion_matrix(labels, preds)

  results = {
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall,
      'f1 measure': f1,
      'confusion matrix': matrix
  }
  return results

In [57]:
def training(model, train_loader, val_loader, criterion, optimizer, epochs, device):
  model.to(device)

  best_acc = 0.0
  for epoch in range(epochs):
    # Poner modelo en modo evaluación
    model.train()

    # Fase entrenamiento
    running_loss = 0.0
    for input, label in train_loader:
      input, label = input.to(device), label.to(device)
      optimizer.zero_grad()
      output = model(input)
      loss = criterion(output, label)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

    # Fase validación
    metrics = evaluation(model, val_loader, device)
    accuracy = metrics['accuracy']

    print(f"Epoch {epoch + 1}/{epochs}|   Train loss: {running_loss/len(train_loader):.4f}|   Val accuracy: {accuracy:.4f}")

    # Guardar el modelo
    if accuracy > best_acc:
      best_acc = accuracy
      torch.save(model.state_dict(), f"best_{type(model).__name__}.pth")

  print(f"Training completado!")

In [58]:
models = {
    'ResNet18': (modelo_resnet, resnet_optimizer),
    'MobileNet': (modelo_mobilenet, mobilenet_optimizer),
    'CNN': (modelo_cnn, cnn_optimizer)
}

for name, (model, optimizer) in models.items():
    print(f"\nTraining {name}")
    training(model, train_loader, val_loader, criterion, optimizer, epochs=10, device=device)


Training ResNet18
Epoch 1/10|   Train loss: 0.0816|   Val accuracy: 0.9333
Epoch 2/10|   Train loss: 0.0259|   Val accuracy: 1.0000
Epoch 3/10|   Train loss: 0.0060|   Val accuracy: 1.0000
Epoch 4/10|   Train loss: 0.0019|   Val accuracy: 0.9667
Epoch 5/10|   Train loss: 0.0011|   Val accuracy: 1.0000
Epoch 6/10|   Train loss: 0.0020|   Val accuracy: 1.0000
Epoch 7/10|   Train loss: 0.0010|   Val accuracy: 1.0000
Epoch 8/10|   Train loss: 0.0008|   Val accuracy: 1.0000
Epoch 9/10|   Train loss: 0.0003|   Val accuracy: 1.0000
Epoch 10/10|   Train loss: 0.0003|   Val accuracy: 1.0000
Training completado!

Training MobileNet
Epoch 1/10|   Train loss: 0.0001|   Val accuracy: 0.9667
Epoch 2/10|   Train loss: 0.0002|   Val accuracy: 0.9667
Epoch 3/10|   Train loss: 0.0008|   Val accuracy: 0.9667
Epoch 4/10|   Train loss: 0.0003|   Val accuracy: 0.9667
Epoch 5/10|   Train loss: 0.0002|   Val accuracy: 0.9667
Epoch 6/10|   Train loss: 0.0002|   Val accuracy: 0.9667
Epoch 7/10|   Train loss: 0

# Evaluation

In [55]:
# Evaluar modelos
models = {
    'ResNet18': modelo_resnet,
    'MobileNetV2': modelo_mobilenet,
    'Custom CNN': modelo_cnn
}

results = {}
for name, model in models.items():
  print(f"Evaluando modelo: {name}")
  model.load_state_dict(torch.load(f"best_{type(model).__name__}.pth"))
  results[name] = evaluation(model, test_loader, device)

# Imprimir resultados
for model, metrics in results.items():
    print(f'\n{model} Performance:')
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1-Score: {metrics['f1 measure']:.4f}")
    print("Confusion Matrix:")
    print(metrics['confusion matrix'])

Evaluando modelo: ResNet18
Evaluando modelo: MobileNetV2
Evaluando modelo: Custom CNN

ResNet18 Performance:
Accuracy: 0.9667
Precision: 0.9685
Recall: 0.9667
F1-Score: 0.9665
Confusion Matrix:
[[12  1]
 [ 0 17]]

MobileNetV2 Performance:
Accuracy: 0.9333
Precision: 0.9422
Recall: 0.9333
F1-Score: 0.9336
Confusion Matrix:
[[13  0]
 [ 2 15]]

Custom CNN Performance:
Accuracy: 0.8667
Precision: 0.8921
Recall: 0.8667
F1-Score: 0.8616
Confusion Matrix:
[[ 9  4]
 [ 0 17]]


# Debug

In [35]:
###DEBUG --> borrar los contenidos de una carpeta###
import os
import shutil

carpeta = "/content/dataset"

for archivo in os.listdir(carpeta):
    ruta_completa = os.path.join(carpeta, archivo)
    if os.path.isfile(ruta_completa) or os.path.islink(ruta_completa):
        os.unlink(ruta_completa)
    elif os.path.isdir(ruta_completa):
        shutil.rmtree(ruta_completa)
